In [ ]:
import os
import json
import csv

def data_parsing(data, writer):
    """
    JSON 데이터에서 korean_text와 gloss_id들을 추출하여 CSV 파일에 기록합니다.
    gloss_id는 sign_gestures_both와 sign_gestures_strong 리스트에서 추출하며,
    여러 gloss_id가 있을 경우 띄어쓰기로 연결합니다.
    
    Args:
        data (dict): JSON 파일에서 로드한 데이터.
        writer (csv.writer): CSV 파일에 행을 기록하기 위한 writer 객체.
    """
    # JSON 최상위에서 korean_text 추출
    korean_text = data.get("korean_text", "")
    
    # gloss_id 추출 (두 개 리스트에서 모두 추출)
    gloss_ids = []
    sign_script = data.get("sign_script", {})
    
    for gesture_type in ["sign_gestures_both", "sign_gestures_strong"]:
        for gesture in sign_script.get(gesture_type, []):
            gloss_id = gesture.get("gloss_id", "")
            if gloss_id:
                gloss_ids.append(gloss_id)
                
    # 여러 gloss_id가 있을 경우 띄어쓰기로 결합
    gloss_ids_str = " ".join(gloss_ids)
    
    # CSV 파일에 한 행 기록: [korean_text, gloss_ids_str]
    writer.writerow([korean_text, gloss_ids_str])

def process_json_files(base_dir, csv_writer):
    """
    지정된 base_dir 하위의 모든 JSON 파일을 순차적으로 읽어,
    각 파일의 데이터를 data_parsing 함수를 통해 CSV에 기록합니다.
    
    Args:
        base_dir (str): JSON 파일들이 포함된 최상위 디렉토리 경로.
        csv_writer (csv.writer): CSV 파일에 행을 기록하기 위한 writer 객체.
    """
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.lower().endswith('.json'):
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                        data_parsing(data, csv_writer)
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")

if __name__ == "__main__":
    base_path = "/Users/kwonsejin/Desktop/side_project/114.재난 안전 정보 전달을 위한 수어영상 데이터/01.데이터/2.Validation/라벨링데이터/03_JSON_VL/1.tact_morpheme/4.기타재난"
    output_csv_path = base_path + "output.csv"
    
    # CSV 파일을 열고 헤더를 작성한 후, JSON 파일들을 순차적으로 처리합니다.
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["korean_text", "gloss_id"])  # CSV 헤더
        process_json_files(base_path, writer)
    
    print("CSV 파일 생성 완료:", output_csv_path)